In [ ]:
import pandas as pd
import numpy as np
from zipfile import ZipFile
from pathlib import Path
import matplotlib.pyplot as plt
import os
import json
import re
import gzip
from urllib.request import urlopen

In [ ]:
!wget http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Cell_Phones_and_Accessories_5.json.gz

--2020-07-29 05:13:07--  http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Cell_Phones_and_Accessories_5.json.gz
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 45409631 (43M) [application/x-gzip]
Saving to: ‘reviews_Cell_Phones_and_Accessories_5.json.gz’

reviews_Cell_Phones 100%[===================>]  43.31M  4.80MB/s    in 9.0s    

2020-07-29 05:13:16 (4.83 MB/s) - ‘reviews_Cell_Phones_and_Accessories_5.json.gz’ saved [45409631/45409631]



In [ ]:
data = []
with gzip.open('reviews_Cell_Phones_and_Accessories_5.json.gz') as f:
    for l in f:
        data.append(json.loads(l.strip()))
print(len(data))
print(data[0])

194439
{'reviewerID': 'A30TL5EWN6DFXT', 'asin': '120401325X', 'reviewerName': 'christina', 'helpful': [0, 0], 'reviewText': "They look good and stick good! I just don't like the rounded shape because I was always bumping it and Siri kept popping up and it was irritating. I just won't buy a product like this again", 'overall': 4.0, 'summary': 'Looks Good', 'unixReviewTime': 1400630400, 'reviewTime': '05 21, 2014'}


In [ ]:
reviews = pd.DataFrame.from_dict(data)
reviews.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A30TL5EWN6DFXT,120401325X,christina,"[0, 0]",They look good and stick good! I just don't li...,4.0,Looks Good,1400630400,"05 21, 2014"
1,ASY55RVNIL0UD,120401325X,emily l.,"[0, 0]",These stickers work like the review says they ...,5.0,Really great product.,1389657600,"01 14, 2014"
2,A2TMXE2AFO7ONB,120401325X,Erica,"[0, 0]",These are awesome and make my phone look so st...,5.0,LOVE LOVE LOVE,1403740800,"06 26, 2014"
3,AWJ0WZQYMYFQ4,120401325X,JM,"[4, 4]",Item arrived in great time and was in perfect ...,4.0,Cute!,1382313600,"10 21, 2013"
4,ATX7CZYFXI1KW,120401325X,patrice m rogoza,"[2, 3]","awesome! stays on, and looks great. can be use...",5.0,leopard home button sticker for iphone 4s,1359849600,"02 3, 2013"


In [ ]:
regex = re.compile('[^a-z]+')
def cleanup(sentence):
    sentence = sentence.lower()
    sentence = regex.sub(' ', sentence).strip()
    sentence = " ".join(nltk.word_tokenize(sentence))
    return sentence

In [ ]:
import nltk
nltk.download('punkt')
reviews["reviewText"] = reviews["reviewText"].apply(cleanup) #reviews["summary"].apply(cleanup)
reviews[:5]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A30TL5EWN6DFXT,120401325X,christina,"[0, 0]",they look good and stick good i just don t lik...,4.0,Looks Good,1400630400,"05 21, 2014"
1,ASY55RVNIL0UD,120401325X,emily l.,"[0, 0]",these stickers work like the review says they ...,5.0,Really great product.,1389657600,"01 14, 2014"
2,A2TMXE2AFO7ONB,120401325X,Erica,"[0, 0]",these are awesome and make my phone look so st...,5.0,LOVE LOVE LOVE,1403740800,"06 26, 2014"
3,AWJ0WZQYMYFQ4,120401325X,JM,"[4, 4]",item arrived in great time and was in perfect ...,4.0,Cute!,1382313600,"10 21, 2013"
4,ATX7CZYFXI1KW,120401325X,patrice m rogoza,"[2, 3]",awesome stays on and looks great can be used o...,5.0,leopard home button sticker for iphone 4s,1359849600,"02 3, 2013"


In [ ]:
count = reviews.groupby("asin", as_index=False).count()
mean = reviews.groupby("asin", as_index=False).mean()

In [ ]:
reviews_temp = pd.merge(reviews, count, how='right', on=['asin'])
reviews_temp = pd.merge(reviews_temp, mean, how='right', on=['asin'])

In [ ]:
reviews_temp[:5]

,reviewerID_x,asin,reviewerName_x,helpful_x,reviewText_x,overall_x,summary_x,unixReviewTime_x,reviewTime_x,reviewerID_y,reviewerName_y,helpful_y,reviewText_y,overall_y,summary_y,unixReviewTime_y,reviewTime_y,overall,unixReviewTime
0,A30TL5EWN6DFXT,120401325X,christina,"[0, 0]",they look good and stick good i just don t lik...,4.0,Looks Good,1400630400,"05 21, 2014",7,7,7,7,7,7,7,7,4.428571,1.384980e+09
1,ASY55RVNIL0UD,120401325X,emily l.,"[0, 0]",these stickers work like the review says they ...,5.0,Really great product.,1389657600,"01 14, 2014",7,7,7,7,7,7,7,7,4.428571,1.384980e+09
2,A2TMXE2AFO7ONB,120401325X,Erica,"[0, 0]",these are awesome and make my phone look so st...,5.0,LOVE LOVE LOVE,1403740800,"06 26, 2014",7,7,7,7,7,7,7,7,4.428571,1.384980e+09
3,AWJ0WZQYMYFQ4,120401325X,JM,"[4, 4]",item arrived in great time and was in perfect ...,4.0,Cute!,1382313600,"10 21, 2013",7,7,7,7,7,7,7,7,4.428571,1.384980e+09
4,ATX7CZYFXI1KW,120401325X,patrice m rogoza,"[2, 3]",awesome stays on and looks great can be used o...,5.0,leopard home button sticker for iphone 4s,1359849600,"02 3, 2013",7,7,7,7,7,7,7,7,4.428571,1.384980e+09


In [ ]:
reviews_temp["Count"] = reviews_temp["reviewerID_y"]
reviews_temp["Score"] = reviews_temp["overall"]
reviews_temp["Summary"] = reviews_temp["reviewText_x"]
reviews_temp["ProductID"] = reviews_temp["asin"]

In [ ]:
reviews_temp = reviews_temp[['ProductID','Summary','Score',"Count"]]

In [ ]:
reviews["ProductID"] = reviews["asin"]
reviews1 = reviews.groupby("ProductID", as_index=False).mean()
combine_summary = reviews_temp.groupby("ProductID")["Summary"].apply(list)
combine_summary = pd.DataFrame(combine_summary)

In [ ]:
combine_summary = pd.merge(combine_summary, reviews_temp, on="ProductID", how='inner')
combine_summary = combine_summary[['ProductID','Summary_x','Score']]

In [ ]:
combine_summary = combine_summary.drop_duplicates(['Score'], keep='last')
combine_summary = combine_summary.reset_index()

In [ ]:
combine_summary["Summary_x"][0]

['before buying this set i searched a lot and did read a lot of reviews and finally i chose to go with this set the screwdrivers are very easy to use specially with iphones i had to change the batterry on my iphone and if stanley screwdriver set was not there i was about to break my iphone thanks for making this set so helpful and easy to use',
 'handy little set to have around if you tinker with electronic devices this set has many uses for otherthings also product is as described and price is right i recommend everyone having a set',
 'i bought this set to fix a pair of eyeglasses that had loose screws the smallest screwdriver in this kit was perfect for that job so mission accomplished the tops of each screwdriver twirls independently from the shaft meaning you can keep a finger on top of the head to stabilize the driver while you re screwing sorry there s no other way to say that the box suggests these screwdrivers can be used for repairing eyeglasses cameras watches fishing reels 

In [ ]:
def extract_candidate_words(text, good_tags=set(['JJ','JJR','JJS','NN','NNP','NNS','NNPS'])):
    import itertools, nltk, string

    punct = set(string.punctuation)
    stop_words = set(nltk.corpus.stopwords.words('english'))
    tagged_words = itertools.chain.from_iterable(nltk.pos_tag_sents(nltk.word_tokenize(sent) for sent in nltk.sent_tokenize(text)))

    candidates = [word.lower() for word, tag in tagged_words
                  if tag in good_tags and word.lower() not in stop_words
                  and not all(char in punct for char in word)]

    return candidates

In [ ]:
review_text = ''
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
for i in combine_summary["Summary_x"][0]:
   review_text = review_text + ' ' + i
#review_text=extract_candidate_words(review_text)
review_text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


' before buying this set i searched a lot and did read a lot of reviews and finally i chose to go with this set the screwdrivers are very easy to use specially with iphones i had to change the batterry on my iphone and if stanley screwdriver set was not there i was about to break my iphone thanks for making this set so helpful and easy to use handy little set to have around if you tinker with electronic devices this set has many uses for otherthings also product is as described and price is right i recommend everyone having a set i bought this set to fix a pair of eyeglasses that had loose screws the smallest screwdriver in this kit was perfect for that job so mission accomplished the tops of each screwdriver twirls independently from the shaft meaning you can keep a finger on top of the head to stabilize the driver while you re screwing sorry there s no other way to say that the box suggests these screwdrivers can be used for repairing eyeglasses cameras watches fishing reels hobbies 

In [ ]:
def score_keyphrases_by_textrank(text, n_keywords=0.05):
    from itertools import takewhile, tee#, izip
    import networkx, nltk
    
    words = [word.lower()
             for sent in nltk.sent_tokenize(text)
             for word in nltk.word_tokenize(sent)]
    candidates = extract_candidate_words(text)
    graph = networkx.Graph()
    graph.add_nodes_from(set(candidates))

    def pairwise(iterable):
        a, b = tee(iterable)
        next(b, None)
        return zip(a, b)
        
    for w1, w2 in pairwise(candidates):
        if w2:
            graph.add_edge(*sorted([w1, w2]))
    ranks = networkx.pagerank(graph)
    if 0 < n_keywords < 1:
        n_keywords = int(round(len(candidates) * n_keywords))
    word_ranks = {word_rank[0]: word_rank[1]
                  for word_rank in sorted(ranks.items(), key=lambda x: x[1], reverse=True)[:n_keywords]}
    keywords = set(word_ranks.keys())
    keyphrases = {}
    j = 0
    for i, word in enumerate(words):
        if i < j:
            continue
        if word in keywords:
            kp_words = list(takewhile(lambda x: x in keywords, words[i:i+10]))
            avg_pagerank = sum(word_ranks[w] for w in kp_words) / float(len(kp_words))
            keyphrases[' '.join(kp_words)] = avg_pagerank
            j = i + len(kp_words)
    
    return sorted(keyphrases.items(), key=lambda x: x[1], reverse=True)

In [ ]:
import datetime
a = datetime.datetime.now() 
lngt = len(score_keyphrases_by_textrank(review_text))
b = datetime.datetime.now() 
print(b-a)

0:00:00.326738


In [ ]:
for i in range(lngt):
  if len(score_keyphrases_by_textrank(review_text)[i][0]) > 10:
    print(score_keyphrases_by_textrank(review_text)[i][0])

iphone battery
plastic tools
good tool set
battery use
screwdriver set
right tools
iphone iphone
handy tools
old battery
battery replacement
battery last
battery fit
tools work great
small electronics great tools
plastic kit
great price
nice quality set
small plastic
good quality
quality great price
small screwdrivers
right tools things
torx driver tools
useful tool
screwdrivers
plastic case
small screws
product great price
use handy little set
hard plastic
small electronics
great value
cheap plastic
small head screwdrivers
screwdriver
good nothing
torx screwdrivers
iphone star screw
quality product
quality handles
head screwdrivers
little better
nice case nice
handy screwdrivers work
little things
smaller screwdriver
electronics
torx drivers
screw drivers
flat head screwdriver
screw right
torx screw drivers work
several sizes
screw drivers work
nothing screw drivers
smaller sizes
screw driver
different sizes
star screw driver
tweezers work
handle tweezers
handle flat driver
several di